# Ensemble Retriever

In [ ]:
!pip install chromadb tiktoken transformers sentence_transformers openai langchain tablib

In [ ]:
!pip install langchain_community

In [ ]:
import os
import openai
os.environ["OPENAI_API_KEY"] = " "

In [ ]:
import tiktoken

#텍스트 토큰으로 분할
tokenizer = tiktoken.get_encoding("cl100k_base")

#토큰 수
def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import CSVLoader

In [ ]:
loader = CSVLoader(file_path=" ")
documents = loader.load()
pages = loader.load_and_split()

In [ ]:
#page별로 split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50, length_function = tiktoken_len)
texts = text_splitter.split_documents(pages)

from langchain.embeddings import HuggingFaceEmbeddings

#허깅페이스 모델을 이용해 임베딩 벡터로 변환
model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

#Chroma Vectorstore에 저장
docsearch = Chroma.from_documents(texts, hf)

In [ ]:
# 앙상블 Retriever
from langchain.retrievers import EnsembleRetriever
from langchain.retrievers.document_compressors import LengthBasedDocumentCompressor
from langchain.retrievers.document_compressors import LLMChainCompressor
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# BM25 리트리버 생성
#k1:용어빈도 중요도, b:문서길이 중요도
bm25_retriever = BM25Retriever(
    collection=docsearch.as_retriever().get_relevant_documents,
    search_kwargs={'k1': 1.2, 'b': 0.75}
)

# 임베딩 기반 (문서 검색) 리트리버 생성
# 문서 검색 기능 제공 객체에서 관련 문서 가져옴
# search_k = 최근접 이웃 수, k=3 = k개의 관련성 높은 문서 반환
faiss_retriever = FaissRetriever(
    collection=docsearch.as_retriever().get_relevant_documents,
    search_k=50
    k=5
)

# 리트리버 리스트 생성
retrievers = [bm25_retriever, faiss_retriever]

# 문서 압축기 생성 (선택 사항)
# compressor = LLMChainCompressor(llm=OpenAI(temperature=0.5))

# Ensemble Retriever 생성
ensemble_retriever = EnsembleRetriever(
    retrievers=retrievers,
    # document_compressor=compressor
)

# Streaming 출력을 위한 콜백 핸들러 생성
callback_handler = StreamingStdOutCallbackHandler()

# RetrievalQA 생성
# "stuff": 검색된 문서를 연결하여 응답 생성 방식
qa = RetrievalQA.from_chain_type(
    llm=OpenAI(temperature=0.5, callbacks=[callback_handler]),
    chain_type="stuff",
    retriever=ensemble_retriever,
    return_source_documents=True  #원본 문서도 함께 반환
)

# 질문 실행
query = ""
result = qa(query)